# CZII Prepare Image Data for UNET

https://www.kaggle.com/code/stpeteishii/czii-zarr-image-view

https://www.kaggle.com/code/stpeteishii/czii-data-3d-view

https://www.kaggle.com/code/stpeteishii/czii-static-image-and-overlay-image

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import gc

In [2]:
# フォルダが存在する場合は削除
if os.path.exists('../proc/prepare_image_data_for_unet/mask'):
    os.system('rm -rf ../proc/prepare_image_data_for_unet/mask')
if os.path.exists('../proc/prepare_image_data_for_unet/image'):  
    os.system('rm -rf ../proc/prepare_image_data_for_unet/image')
if os.path.exists('../proc/prepare_image_data_for_unet/timage'):
    os.system('rm -rf ../proc/prepare_image_data_for_unet/timage')

# フォルダが存在しない場合のみ作成
if not os.path.exists('../proc/prepare_image_data_for_unet/image'):
    os.makedirs('../proc/prepare_image_data_for_unet/image')
if not os.path.exists('../proc/prepare_image_data_for_unet/mask'):
    os.makedirs('../proc/prepare_image_data_for_unet/mask') 
if not os.path.exists('../proc/prepare_image_data_for_unet/timage'):
    os.makedirs('../proc/prepare_image_data_for_unet/timage')

MASK_DIR = '../proc/prepare_image_data_for_unet/mask'
IMAGE_DIR = '../proc/prepare_image_data_for_unet/image'
TIMAGE_DIR = '../proc/prepare_image_data_for_unet/timage'

# !rm -rf mask
# !rm -rf image
# !rm -rf timage

# !mkdir image
# !mkdir mask
# !mkdir timage

In [3]:
dir0="../raw/train/overlay/ExperimentRuns/"
names=os.listdir(dir0)
TSpaths=[]
for name in names:
    TSpaths+=[os.path.join(dir0,name)]
print(TSpaths)

PATHS=[]
for TSpath in TSpaths:
    paths=[]
    for dirname, _, filenames in os.walk(TSpath):
        for filename in filenames:
            paths+=[(os.path.join(dirname, filename))]
    PATHS+=[paths]

['../raw/train/overlay/ExperimentRuns/TS_5_4', '../raw/train/overlay/ExperimentRuns/TS_69_2', '../raw/train/overlay/ExperimentRuns/TS_6_4', '../raw/train/overlay/ExperimentRuns/TS_6_6', '../raw/train/overlay/ExperimentRuns/TS_73_6', '../raw/train/overlay/ExperimentRuns/TS_86_3', '../raw/train/overlay/ExperimentRuns/TS_99_9']


In [4]:
import json

def load_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

from collections import defaultdict
cnt = defaultdict(int)

In [5]:
LL=[]
for paths in PATHS:
    L=[]
    for i,path in enumerate(paths):
        data=load_from_json(path)
        cnt[i]=data['pickable_object_name']
        X=[]
        Y=[]
        Z=[]
        Z2=[]
        for datai in data['points']:
            xyz=datai['location']
            X+=[xyz['x']]
            Y+=[xyz['y']]
            Z+=[xyz['z']]
            Z2+=[xyz['z']//50]
        L+=[(i,X,Y,Z,Z2)]
    LL+=[L]

---

# create mask images

---

In [6]:
def z_filtered(L,k,j):
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    
    for data in L:
        i, X, Y, Z, Z2 = data 
        filtered_indices = [idx for idx, z in enumerate(Z2) if z==j]
        
        # Apply the filter to X, Y, and Z2
        filtered_X = [X[idx] for idx in filtered_indices]
        filtered_Y = [Y[idx] for idx in filtered_indices]
        filtered_Z2 = [Z2[idx] for idx in filtered_indices]
        
        # Plot the filtered data
        scatter = ax.scatter(filtered_X, filtered_Y, c=[i] * len(filtered_X),
                             vmin=0, vmax=5, cmap='viridis')
    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    #cbar = fig.colorbar(scatter, ax=ax)
    #cbar.set_label('Label')
    plt.axis('off')
    plt.savefig(f'{MASK_DIR}/{k}_{j:02}.png', bbox_inches='tight', pad_inches=0)
    
    plt.close()

In [7]:
for k,L in enumerate(LL):
    for j in range(35):
        print(k,j)
        z_filtered(L,k,j)
        print()
        print('-----------------------------'*2)
        print()

gc.collect()

0 0

----------------------------------------------------------

0 1

----------------------------------------------------------

0 2

----------------------------------------------------------

0 3

----------------------------------------------------------

0 4

----------------------------------------------------------

0 5

----------------------------------------------------------

0 6

----------------------------------------------------------

0 7

----------------------------------------------------------

0 8

----------------------------------------------------------

0 9



----------------------------------------------------------

0 10

----------------------------------------------------------

0 11

----------------------------------------------------------

0 12

----------------------------------------------------------

0 13

----------------------------------------------------------

0 14

----------------------------------------------------------

0 15

----------------------------------------------------------

0 16

----------------------------------------------------------

0 17

----------------------------------------------------------

0 18

----------------------------------------------------------

0 19

----------------------------------------------------------

0 20

----------------------------------------------------------

0 21

----------------------------------------------------------

0 22

----------------------------------------------------------

0 23

----------------------------------------------------------

0 24

---------

6

---

# create train images

---

In [8]:
# !pip install zarr

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
import zarr
from PIL import Image

In [10]:
dir1='../raw/train/static/ExperimentRuns'

names=os.listdir(dir1)
Spaths=[]
for name in names:
    Spaths+=[os.path.join(dir1,name)]
print(Spaths)

ZARRS=[]
for sp in Spaths:
    files=os.listdir(os.path.join(sp,'VoxelSpacing10.000'))
    ZARR=[]
    for file in files:
        ZARR+=[os.path.join(sp,'VoxelSpacing10.000',file)]
    ZARRS+=[ZARR]

for k,ZARR in enumerate(ZARRS):
    for j,diri in enumerate(ZARR[0:1]):
        print(ZARR[j].split('/')[-1]) 
        data = zarr.open(diri, mode='r') 
        fig = plt.figure(figsize=(12,18))
    
        for i in range(35):
            ax = plt.subplot(7, 5, i + 1)
            plt.axis('off')
            image=data[2][i]
            min_val, max_val = image.min(), image.max()
            simage = ((image - min_val) / (max_val - min_val) * 255).astype('uint8')
            plt.imshow(simage)
            cv2.imwrite(f'{IMAGE_DIR}/{k}_{i:02}.png', simage)
        plt.close()

gc.collect()

['../raw/train/static/ExperimentRuns/TS_5_4', '../raw/train/static/ExperimentRuns/TS_69_2', '../raw/train/static/ExperimentRuns/TS_6_4', '../raw/train/static/ExperimentRuns/TS_6_6', '../raw/train/static/ExperimentRuns/TS_73_6', '../raw/train/static/ExperimentRuns/TS_86_3', '../raw/train/static/ExperimentRuns/TS_99_9']
ctfdeconvolved.zarr


ctfdeconvolved.zarr
ctfdeconvolved.zarr
ctfdeconvolved.zarr
ctfdeconvolved.zarr
ctfdeconvolved.zarr
ctfdeconvolved.zarr


33

---

# create test images

---

In [11]:
dir2='../raw/test/static/ExperimentRuns'

names=os.listdir(dir2)
tSpaths=[]
for name in names:
    tSpaths+=[os.path.join(dir2,name)]
print(tSpaths)

tZARRS=[]
for sp in tSpaths:
    files=os.listdir(os.path.join(sp,'VoxelSpacing10.000'))
    tZARR=[]
    for file in files:
        tZARR+=[os.path.join(sp,'VoxelSpacing10.000',file)]
    tZARRS+=[tZARR]

for k,tZARR in enumerate(tZARRS):
    for j,diri in enumerate(tZARR[0:1]):
        print(tZARR[j].split('/')[-1]) 
        data = zarr.open(diri, mode='r') 
        fig = plt.figure(figsize=(12,18))
    
        for i in range(35):
            ax = plt.subplot(7, 5, i + 1)
            plt.axis('off')
            image=data[2][i]
            min_val, max_val = image.min(), image.max()
            simage = ((image - min_val) / (max_val - min_val) * 255).astype('uint8')
            plt.imshow(simage)
            cv2.imwrite(f'{TIMAGE_DIR}/{k}_{i:02}.png', simage)
        plt.close()

['../raw/test/static/ExperimentRuns/TS_5_4', '../raw/test/static/ExperimentRuns/TS_69_2', '../raw/test/static/ExperimentRuns/TS_6_4']
denoised.zarr


denoised.zarr
denoised.zarr
